In [1]:
from gridworld import *
import numpy as np


In [2]:
world0 = gridworld((5,5),(2,4),[(2,2),(2,0)],[(1,1),(2,1),(1,3),(2,3)],[(4,0),(4,1),(4,2),(4,3),(4,4)],p_e=0.2,dest_rewards=[1,10],haz_rewards=-1,dest_names=['diddy reese','safron rose'])
world1 = gridworld((5,5),(2,4),[(2,2),],[(1,1),(2,1),(1,3),(2,3)],[(4,0),(4,1),(4,2),(4,3),(4,4)],p_e=0.2,dest_rewards=[1,],haz_rewards=-1,dest_names=['diddy reese',])
world2 = gridworld((5,5),(2,4),[(2,0),],[(1,1),(2,1),(1,3),(2,3)],[(4,0),(4,1),(4,2),(4,3),(4,4)],p_e=0.2,dest_rewards=[10,],haz_rewards=-1,dest_names=['safron rose',])

world1.calc_p_matrix()
world1.calc_r_matrix()
world2.calc_p_matrix()
world2.calc_r_matrix()

states = np.arange(world0.num_states)
action_indices = np.arange(len(actions))

AttributeError: 'gridworld' object has no attribute 'n_states'

In [45]:
def transition_model(world, state, action, state_next):
    if state < world.num_states and state_next < world.num_states and action < 5:
        return world.p_matrix[action, state, state_next]
    else:
        print('Invalid state or action')
        return None

def reward_function(world, state, action, state_next):
    if state < world.num_states and state_next < world.num_states and action < 5:
        return world.r_matrix[action, state, state_next]
    else:
        print('Invalid state or action')
        return None

def value_iteration(world, gamma, epsilon):
    states = np.arange(world.num_states)
    # Initialize value function
    value_map = {s: 0 for s in states}
    action_indices = np.arange(len(actions))

    while True:
        delta = 0
        for s in states:
            v = value_map[s]
            value_map[s] = max(sum(transition_model(world,s, a, s_next) * (reward_function(world, s, a, s_next) + gamma * value_map[s_next])
                            for s_next in states) for a in action_indices) # value function
            delta = max(delta, abs(v - value_map[s])) # amount in which value function has changed
        
        # Check for convergence
        if delta < epsilon:
            break
    
    return value_map

def policy_extraction(world, value_map, gamma):
    policy = {}
    for s in states:
        # find the max a in actions that maximize the function given by lambda
        policy[s] = max(action_indices, key=lambda a: sum(transition_model(world, s, a, s_next) * (reward_function(world, s, a, s_next) + gamma * value_map[s_next])
                                                    for s_next in states))
    return policy

def get_policy_map(world, policy):
    policy_map = np.zeros((world.x_max,world.y_max),dtype = str)
    action_signs = ['o','→','←','↑','↓']
    for i in range(world.y_max):
        for j in range(world.x_max):
            policy_map[world.y_max -1 -i][j] = action_signs[policy[i*world.x_max+j]]
    print(policy_map)
    return policy_map

In [46]:
value_map1 = value_iteration(world1, 0.8, 0.01)
value_map2 = value_iteration(world2, 0.8, 0.01)
policy1 = policy_extraction(world1, value_map1, 0.8)
policy2 = policy_extraction(world2, value_map2, 0.8)

In [47]:
value_map = {s: value_map1[s] + value_map2[s] for s in states}
policy = policy_extraction(world1, value_map, 0.8)

In [48]:
policy_map1 = get_policy_map(world1, policy1)
print()
policy_map2 = get_policy_map(world2, policy2)
print()
policy_map = get_policy_map(world0, policy)

[['↓' '←' '→' '↓' '←']
 ['↓' 'o' 'o' '↓' '←']
 ['→' '→' 'o' '←' '←']
 ['↑' 'o' 'o' '↑' '←']
 ['↑' '←' '→' '↑' '←']]

[['↓' '←' '→' '↓' '←']
 ['↓' 'o' 'o' '↓' '←']
 ['↓' '←' '→' '↓' '↓']
 ['↓' 'o' 'o' '↓' '←']
 ['→' '→' 'o' '←' '←']]

[['↓' '←' '→' '↓' '←']
 ['↓' 'o' 'o' '↓' '←']
 ['↓' '→' '→' '↓' '←']
 ['↓' 'o' 'o' '↓' '←']
 ['→' '→' 'o' '←' '←']]
